## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps


# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current description
0,0,Puerto Ayora,EC,-0.7393,-90.3518,66.67,94,100,6.46,overcast clouds
1,1,Mataura,NZ,-46.1927,168.8643,51.48,68,75,7.47,broken clouds
2,2,Avarua,CK,-21.2078,-159.7750,78.85,89,40,8.05,light rain
3,3,Kieta,PG,-6.2167,155.6333,80.89,85,100,2.53,light rain
4,4,Atar,MR,20.5169,-13.0499,65.21,12,87,9.42,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp=float(input("What is your minium temperature you would like for your trip? "))
max_temp=float(input("What is your mixium temperature you would like for your trip? "))


What is your minium temperature you would like for your trip? 40
What is your mixium temperature you would like for your trip? 85


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current description
0,0,Puerto Ayora,EC,-0.7393,-90.3518,66.67,94,100,6.46,overcast clouds
1,1,Mataura,NZ,-46.1927,168.8643,51.48,68,75,7.47,broken clouds
2,2,Avarua,CK,-21.2078,-159.7750,78.85,89,40,8.05,light rain
3,3,Kieta,PG,-6.2167,155.6333,80.89,85,100,2.53,light rain
4,4,Atar,MR,20.5169,-13.0499,65.21,12,87,9.42,overcast clouds
5,5,Punta Arenas,CL,-53.1500,-70.9167,43.29,75,100,6.49,overcast clouds
6,6,Codrington,AU,-38.2667,141.9667,62.94,64,2,14.14,clear sky
8,8,Durango,MX,24.8333,-104.8333,55.49,37,100,5.37,overcast clouds
9,9,Hermanus,ZA,-34.4187,19.2345,66.90,81,100,6.44,overcast clouds
10,10,Albany,US,42.6001,-73.9662,44.31,75,100,1.99,overcast clouds


In [7]:
preferred_cities_df.count()

City_ID                431
City                   431
Country                428
Lat                    431
Lng                    431
Max Temp               431
Humidity               431
Cloudiness             431
Wind Speed             431
Current description    431
dtype: int64

In [8]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                3
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current description    0
dtype: int64

In [9]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df=preferred_cities_df.dropna()
clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current description    0
dtype: int64

In [10]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current description,Lat,Lng,Hotel Name
0,Puerto Ayora,EC,66.67,overcast clouds,-0.7393,-90.3518,
1,Mataura,NZ,51.48,broken clouds,-46.1927,168.8643,
2,Avarua,CK,78.85,light rain,-21.2078,-159.7750,
3,Kieta,PG,80.89,light rain,-6.2167,155.6333,
4,Atar,MR,65.21,overcast clouds,20.5169,-13.0499,
5,Punta Arenas,CL,43.29,overcast clouds,-53.1500,-70.9167,
6,Codrington,AU,62.94,clear sky,-38.2667,141.9667,
8,Durango,MX,55.49,overcast clouds,24.8333,-104.8333,
9,Hermanus,ZA,66.90,overcast clouds,-34.4187,19.2345,
10,Albany,US,44.31,overcast clouds,42.6001,-73.9662,


In [11]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
   
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [15]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df=hotel_df.loc[(hotel_df["Hotel Name"]!="")]
clean_hotel_df.head()

,City,Country,Max Temp,Current description,Lat,Lng,Hotel Name
0,Puerto Ayora,EC,66.67,overcast clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
1,Mataura,NZ,51.48,broken clouds,-46.1927,168.8643,Ellie's Villa
2,Avarua,CK,78.85,light rain,-21.2078,-159.7750,Paradise Inn
3,Kieta,PG,80.89,light rain,-6.2167,155.6333,Uruna Bay Retreat
4,Atar,MR,65.21,overcast clouds,20.5169,-13.0499,Oumou Elghoura


In [18]:
# 8a. Create the output File (CSV)
output_data_file="WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [19]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [23]:
# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key= g_key)

In [24]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure

fig

Figure(layout=FigureLayout(height='420px'))